In [1]:
from utils.preprocessing import preprocess_videos

In [3]:
datasets = {
    "train": ["train"], # Train dataset folders, now points to sample files
    "test": ["test"], # Test dataset fodler, not points to sample files
}

# When processing full directories remove skip_sampling=True because we don't need 50/50 split in the example.
preprocess_videos(datasets, save_dir="data", video_dir="sample", skip_sampling=True)

Processing train datasets
	 Processing train


11it [02:21, 12.90s/it]


Processing test datasets
	 Processing test


10it [02:23, 14.30s/it]
